In [106]:
from parsetrees.expr_parser import webwork_parser
from parsetrees.expr_parser import Eval_parsed
import pickle
from collections import deque
from collections import defaultdict
import pandas as pd

In [133]:
### Load Week2 data ###
data_week2 = pickle.load(open('../2015data/Week2_data.pkl','rb'))
pd.DataFrame(data_week2[:5])

,answer,attempt,part_id,problem_id,score,set_id,timestamp,user_id,var
0,"{1,2,3,4,5,6}",{},1,1,0,Week2,2015-10-02 00:22:51,zig006,"[($Ac, 0), ($rs, 0), ($A, 0), ($min, 3), ($max..."
1,"{3,5,7,9,11}","{3,5,7,9,11}",2,1,1,Week2,2015-10-02 00:22:51,zig006,"[($Ac, 0), ($rs, 0), ($A, 0), ($min, 3), ($max..."
2,"{1,2,3,4,5,6}","{1,2,3,4,5,6}",1,1,1,Week2,2015-10-02 00:23:41,zig006,"[($Ac, 0), ($rs, 0), ($A, 0), ($min, 3), ($max..."
3,{0},{0},1,2,1,Week2,2015-10-02 00:24:52,zig006,[]
4,{1},"{0,2,3}",2,2,0,Week2,2015-10-02 00:24:52,zig006,[]


In [134]:
### Load Week7a data ###
data_week7a = pickle.load(open('../2015data/Week7a_data.pkl','rb'))
pd.DataFrame(data_week7a[:5])

,answer,attempt,part_id,problem_id,score,set_id,timestamp,user_id,var
0,C,C,1,1,1,Week7a,2015-11-07 01:10:59,dtea,[]
1,A,A,1,1,1,Week7a,2015-11-07 03:22:58,quwong,[]
2,-0.02-2/[sqrt(25)]*2,yes,1,5,0,Week7a,2015-11-07 03:30:22,quwong,"[($n, 25), ($sampleMean, -0.02), ($variance, 2..."
3,-0.02+2/[sqrt(25)]*2,yes,2,5,0,Week7a,2015-11-07 03:30:22,quwong,"[($n, 25), ($sampleMean, -0.02), ($variance, 2..."
4,-1,yes,3,5,0,Week7a,2015-11-07 03:30:22,quwong,"[($n, 25), ($sampleMean, -0.02), ($variance, 2..."


In [138]:
### Group by problem_id and part_id
data = data_week7a
problem_part_collection = defaultdict(list)
for d in data:
    if d['answer']:
        problem_part_collection[(d['problem_id'], d['part_id'])] += [d]

In [49]:
def flatten(tree,tag):
    """flatten a tree
    tag 'c' if it's answer
    tag 't' if it's attempt"""
    List=[]
    Queue=deque([tree])
    try:
        while Queue:
            current=Queue.popleft()
            if type(current)==list: 
                if isinstance(current[0],basestring):
                    List.append([current[1],tag,current])
                elif isinstance(current[0][0],basestring) and isinstance(current[0][1],(int, long, float, complex)) and type(current[0][2])==list:
                    List.append([current[0][1],tag,current])
                    Queue.extend(current[1:])
                else:
                    return []
            else:
                return []
    except Exception as error:
        print 'error: current=', str(current)
        return []
    List=sorted(List,key=lambda x: x[0])
    return List

In [79]:
def find_Hits(List,tol = 1+1e-6):
    """ Given a combined list of subtrees from both attempt and answer,
    sorted by value, find the matching pairs of trees
    tol is the tolerance used to define which pairs of values match. 
    Needed because different expressions get different roundoff error
    """
    Hits=[]
    for i in xrange(0,len(List)-1):
        item1 = List[i]
        item2 = List[i+1]
        if item2[0] != 0:
            ratio=item1[0]/item2[0]
        else:
            ratio = 0
        if item1[1]!=item2[1] and ratio < tol and ratio > (1/tol):
            if item1[1]=='c':
                Hits.append((item1,item2))
            elif item2[1]=='c':
                Hits.append((item2,item1))
            else:
                print "Error in find_Hits. Neither item labeled c"
    return Hits

In [5]:
def find_matches(params):
    attempt=params['attempt']
    attempt_tree=params['att_tree']
    attempt_list=flatten(attempt_tree,'t')
    
    answer=params['answer']
    answer_tree=params['ans_tree']
    answer_list=flatten(answer_tree,'c')

    # sort by value
    combined_list=sorted(answer_list+attempt_list,key=lambda x: x[0])

    Hits=find_Hits(combined_list)

    final_matches=find_dominating_hits(Hits,answer,attempt)
    
    return final_matches

In [7]:
def find_dominating_hits(Hits,answer,attempt):
    """eliminate dominated hits from input Hits"""
    for i in range(len(Hits)):
        for j in range(len(Hits)):
            if i==j:
                continue
            span1=get_span(Hits[i][0])
            span2=get_span(Hits[j][0])
            # If it is dominated by other hits, marked as 'dc'
            if span1[0]<=span2[0] and span1[1]>=span2[1]:
                Hits[j][0][1]='dc'
    
    final_matches=[]
    for hit in Hits:
        # Do not include hits marked as 'dc'
        if hit[0][1]=='c':
            value=hit[0][0]
            span_c=get_span(hit[0])
            span_a=get_span(hit[1])
            answer_part=answer[span_c[0]:span_c[1]+1]
            attempt_part = attempt[span_a[0]:span_a[1]+1]
            if hit[0][2][0]=='X':
                node=hit[0][2][3]
            else:
                node=hit[0][2][0][3]
            final_matches.append((node,value,answer_part,attempt_part))
    return final_matches

In [135]:
def get_span(tree):
    """return the starting index and ending index
        used to determine dominating hits"""
    if tree[2][0]=='X':
        return tree[2][2]
    elif type(tree[2][0])==list:
        return tree[2][0][2]
    else:
        print 'Error in get_span'
        return None

In [136]:
######## Make a list of dictionary ##########
def make_params(problem_id, part_id, problem_part_collection):
    """return list of dict with format
    'user_id':d['user_id'],'answer':d['answer'], 'attempt':d['attempt'],
    'ans_tree':eval_tree,'att_tree':eval_tree_att"""
    params = []
    count = 0
    for d in problem_part_collection[problem_id,part_id]:
        print count
        count += 1
        if d['answer']:
            parse_tree = webwork_parser.parse_webwork(d['answer'])
            if parse_tree == None:
                print 'parse_tree empty'
                continue
            eval_tree = Eval_parsed.eval_parsed(parse_tree[0])
            if eval_tree == None:
                print 'eval_tree empty'
                continue

            parse_tree_att = webwork_parser.parse_webwork(d['attempt'])
            if parse_tree_att == None:
                print 'parse_tree empty'
                continue
            eval_tree_att = Eval_parsed.eval_parsed(parse_tree_att[0])
            if eval_tree_att == None:
                print 'eval_tree empty'
                continue

            params += [{'user_id':d['user_id'], 'answer':d['answer'], 'attempt':d['attempt'],
                        'ans_tree':eval_tree, 'att_tree':eval_tree_att}]
        else:
            print 'no answer'
    return params

In [86]:
### Make params as inputs to function find_matches ###
problem_id = '6'
part_id = '3'
params = make_params(problem_id, part_id, problem_part_collection)
len(params)

608

In [87]:
### Create clusters, if no hits, clustered as 'Nothing' ###
clusters = defaultdict(list)
for p in params:
    final_pairs = find_matches(p)
    if len(final_pairs)>0:
        sorted_final = sorted(final_pairs,key=lambda x: x[0])
        clusters[str(sorted_final)] += [p['attempt']]
    else:
        clusters['Nothing'] += [p['attempt']]

In [99]:
### Print some clusters ###
for c in clusters.keys():
    if len(clusters[c]) > 10:
        print len(clusters[c]), c

17 [('R.1.0.0.1.0.0.0', 0.46153846153846156, u'30/65', u'30/65'), ('R.1.0.1.1.0.0.0', 0.3076923076923077, u'20/65', u'20/65'), ('R.1.0.1.1.1', 2.0, u'2', u'2'), ('R.1.1.1.0.0.0', 0.23076923076923078, u'15/65', u'15/65')]
220 Nothing
11 [('R.1.0.0.1.0.0.0.0', 30.0, u'30', u'30'), ('R.1.0.1.1.0.0.0.0', 20.0, u'20', u'20'), ('R.1.0.1.1.0.0.0.1', 65.0, u'65', u'65'), ('R.1.0.1.1.0.0.1', 0.3333333333333333, u'1/3', u'1/3'), ('R.1.0.1.1.1', 2.0, u'2', u'2'), ('R.1.1.1.0.0.0.0', 15.0, u'15', u'15')]
15 [('R.1.0.0.1.0.0', 0.12820512820512825, u'30/65-1/3', u'(30/65-1/3)/1'), ('R.1.0.1.1.0.0', -0.025641025641025605, u'20/65-1/3', u'(20/65-1/3)/1'), ('R.1.0.1.1.1', 2.0, u'2', u'2'), ('R.1.1.1.0.0', -0.10256410256410253, u'15/65-1/3', u'(15/65-1/3)/1')]


In [ ]:
################# TESTING BELOW ################

In [140]:
#### Example of flatten ####
parse_tree = webwork_parser.parse_webwork('C(48,5)-3*80')
etree = Eval_parsed.eval_parsed(parse_tree[0])
flatten(etree,'t')

[[3.0, 't', ['X', 3.0, [8, 8], 'R.1.0']],
 [5.0, 't', ['X', 5.0, [5, 5], 'R.0.1']],
 [48.0, 't', ['X', 48.0, [2, 3], 'R.0.0']],
 [80.0, 't', ['X', 80.0, [10, 11], 'R.1.1']],
 [240.0,
  't',
  [['*', 240.0, [8, 11], 'R.1'],
   ['X', 3.0, [8, 8], 'R.1.0'],
   ['X', 80.0, [10, 11], 'R.1.1']]],
 [1712064.0,
  't',
  [['-', 1712064.0, [0, 11], 'R'],
   [['C', 1712304, [0, 6], 'R.0'],
    ['X', 48.0, [2, 3], 'R.0.0'],
    ['X', 5.0, [5, 5], 'R.0.1']],
   [['*', 240.0, [8, 11], 'R.1'],
    ['X', 3.0, [8, 8], 'R.1.0'],
    ['X', 80.0, [10, 11], 'R.1.1']]]],
 [1712304,
  't',
  [['C', 1712304, [0, 6], 'R.0'],
   ['X', 48.0, [2, 3], 'R.0.0'],
   ['X', 5.0, [5, 5], 'R.0.1']]]]

In [67]:
#### Example of find_Hits
parse_tree = webwork_parser.parse_webwork('C(48,5)-3*80')
etree_a = Eval_parsed.eval_parsed(parse_tree[0])
parse_tree_b = webwork_parser.parse_webwork('C(48,5)-3*60')
etree_b = Eval_parsed.eval_parsed(parse_tree_b[0])
p = {'attempt':'C(48,5)-3*60', 'answer':'C(48,5)-3*80', 'att_tree':etree_b, 'ans_tree': etree_a}
attempt=p['attempt']
attempt_tree=p['att_tree']
attempt_list=flatten(attempt_tree,'t')

answer=p['answer']
answer_tree=p['ans_tree']
answer_list=flatten(answer_tree,'c')

combined_list=sorted(answer_list+attempt_list,key=lambda x: x[0])

Hits = find_Hits(combined_list)
Hits

[([3.0, 'c', ['X', 3.0, [8, 8], 'R.1.0']],
  [3.0, 't', ['X', 3.0, [8, 8], 'R.1.0']]),
 ([5.0, 'c', ['X', 5.0, [5, 5], 'R.0.1']],
  [5.0, 't', ['X', 5.0, [5, 5], 'R.0.1']]),
 ([48.0, 'c', ['X', 48.0, [2, 3], 'R.0.0']],
  [48.0, 't', ['X', 48.0, [2, 3], 'R.0.0']]),
 ([1712304,
   'c',
   [['C', 1712304, [0, 6], 'R.0'],
    ['X', 48.0, [2, 3], 'R.0.0'],
    ['X', 5.0, [5, 5], 'R.0.1']]],
  [1712304,
   't',
   [['C', 1712304, [0, 6], 'R.0'],
    ['X', 48.0, [2, 3], 'R.0.0'],
    ['X', 5.0, [5, 5], 'R.0.1']]])]

In [141]:
#### Example of find_dominating_hits ####
find_dominating_hits(Hits,answer,attempt)

[('R.1.0', 3.0, '3', '3'), ('R.0', 1712304, 'C(48,5)', 'C(48,5)')]

In [112]:
### Count the number attempts and answer for each part ###
problem_count = defaultdict(int)
problem_part_answer = defaultdict(str)
for d in data:
    if d['answer']:
        problem_count[(d['problem_id'], d['part_id'])] += 1
        problem_part_answer[(d['problem_id'], d['part_id'])] = d['answer']
print len(data)
problem_part_answer

17828


defaultdict(str,
            {(u'1', u'1'): u'B',
             (u'2', u'1'): u'0.05/[sqrt(0.16/200)]',
             (u'2', u'2'): u'Q(1.76777)',
             (u'2',
              u'3'): u'<pre style="text-align:left; padding-left:.2em">yes</pre>',
             (u'2',
              u'4'): u'<pre style="text-align:left; padding-left:.2em">no</pre>',
             (u'5', u'1'): u'-0.02-2/[sqrt(25)]*2',
             (u'5', u'2'): u'-0.03+2/[sqrt(25)]*2',
             (u'5', u'3'): u'-1',
             (u'5', u'4'): u'1',
             (u'5', u'5'): u'1',
             (u'6', u'1'): u'65',
             (u'6',
              u'2'): u'65*(0.5*[(30/65-0.5)/0.5]^2+0.25*[(20/65-0.25)/0.25]^2+0.25*[(15/65-0.25)/0.25]^2)',
             (u'6',
              u'3'): u'65*(1/3*[(30/65-1/3)/(1/3)]^2+1/3*[(20/65-1/3)/(1/3)]^2+1/3*[(15/65-1/3)/(1/3)]^2)',
             (u'6',
              u'4'): u'<pre style="text-align:left; padding-left:.2em">no</pre>',
             (u'6',
              u'5'): u'<pre style=